* [Home](Home.ipynb)
* [Algorithms in Cryptography](Crypto.ipynb)


# Fun with Fermat 

![Pierre de Fermat](https://ingrangel.net/wp-content/uploads/2019/01/image-338.jpg)

Fermat is best known for [his famous last theorem](https://www.nsf.gov/discoveries/disc_summ.jsp?cntn_id=100029&org=NSF), which has been regarded as proved only recently, in 1994 by Andrew Wiles and Richard Taylor, meaning Fermat's proof, if he ever had one, is lost to history.  

In light of the complexity of the existing proof, most are skeptical that he ever had one. He claimed he did, but it wouldn't fit in the margin. "Yeah right" right?

### Fermat's Little Theorem

However his so-called Little Theorem is likewise easy to state and, unlike the Last Theorem, relatively easy to prove.

Statement:

If $p$ is a prime and $a$ is any integer not divisible by p, then $a^{p − 1} − 1$ is divisible by p.

Put another way:

$a^{p − 1} \equiv 1\mod{p}$, where $a \nmid p$.

As long as $a$ and $p$ are strangers, and $p$ is prime, raising $a$ to the $p - 1$ power begets a number divisible by $p$ but for a remainder of 1.

In [1]:
import primes
p = primes.primes_gen.PrimeNumbers()
some_primes = [next(p) for _ in range(20)]

In [2]:
some_primes

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]

In [3]:
def gcd(a, b):
    while a % b:         # when remainder is 0, b is gcd
        b, a = a % b, b  # chopping down to 1
    return b

def strangers(a, b):
    """returns True when a, b have nothing in common"""
    return gcd(a, b) == 1

$p$ divides $a^{p-1} -1$ with no remainder, hence all these zeroes:

The code cell below continues with list comprehension syntax, by adding an optional if clause, a filter.

In [4]:
a = 2
[(pow(a, p-1) - 1) % p for p in some_primes if strangers(a, p)]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Computing $a^{p-1} \mod{p}$ directly is faster, using ```pow``` (which takes a 3rd argument, the modulus) and nets us that residue of one, every time.

In [5]:
[pow(a, p-1, p) for p in some_primes if strangers(a,p)]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

### Fermat's Primality Test 

Given all of the above, i.e. the complete reliability of this theorem for primes (no proof yet), one might think plugging in arbitrary values for p and keeping only those for which the "Fermat Test" holds true, would be a recipie for getting primes.

Lets get all the odd numbers between 3 and 101 and see which, according to the Fermat Test, might be prime.  Since we'll use a = 5 for our base, let's start by weeding out any multiples of 5 as well.

In [6]:
candidates = [c for c in range(3,102,2) if c != 5 and strangers(c, 5)]
print(candidates)

[3, 7, 9, 11, 13, 17, 19, 21, 23, 27, 29, 31, 33, 37, 39, 41, 43, 47, 49, 51, 53, 57, 59, 61, 63, 67, 69, 71, 73, 77, 79, 81, 83, 87, 89, 91, 93, 97, 99, 101]


In [7]:
a = 5
passed = list(filter( lambda p: pow(a, p-1, p) == 1, candidates))
print(passed)

[3, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101]


In [8]:
print([primes.isprime(c) for c in passed])

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


OMG, we got every one of these right!  Does that mean the Fermat test is always reliable?

### Carmichael Numbers

Enter the Carmichael Numbers...

In [9]:
candidates = [c for c in range(501,602,2) if strangers(c, 5)]
print(candidates)

[501, 503, 507, 509, 511, 513, 517, 519, 521, 523, 527, 529, 531, 533, 537, 539, 541, 543, 547, 549, 551, 553, 557, 559, 561, 563, 567, 569, 571, 573, 577, 579, 581, 583, 587, 589, 591, 593, 597, 599, 601]


In [10]:
a = 5
passed = list(filter( lambda p: pow(a, p-1, p) == 1, candidates))
print(passed)

[503, 509, 521, 523, 541, 547, 557, 561, 563, 569, 571, 577, 587, 593, 599, 601]


In [11]:
print([primes.isprime(c) for c in passed])

[True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True]


Oh no, we got a False positive.  The number in that location is not actually prime. What is it?

In [12]:
passed[7]

561

In [13]:
primes.factors(561)

(1, 3, 11, 17)

Would a different base allow the same imposter (not a prime) to pass?  Lets try.

In [14]:
a = 7
candidates = [c for c in range(501,602,2) if strangers(c, a)]

In [15]:
passed = list(filter( lambda p: pow(a, p-1, p) == 1, candidates))
print(passed)

[503, 509, 521, 523, 541, 547, 557, 561, 563, 569, 571, 577, 587, 593, 599, 601]


The same 561 (not a prime) is still there, and turns out to be the first of a special set known as a Carmichael Numbers.  No matter the base, these numbers always pass the Fermat Test.

Does this mean that some composites might pass the Fermat Test for one base, but not another, always preserving the stipulation that the base and the candidate are strangers?  Investigate.  Searching the internet is fine.

Questions / Activities:

* Do some composites pass the Fermat Test that are not Carmichael Numbers?
* What are some of the Carmichael Numbers after 561?  
* Explore between 1000 and 2000.
* Check OEIS for a listing.